In [1]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import Dataset
import torch
import evaluate
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
model_name = 'facebook/mbart-large-50-many-to-many-mmt'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [4]:
source_lang = "en_XX"
target_lang = "fa_IR"

tokenizer.src_lang = source_lang
tokenizer.tgt_lang = target_lang

In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [6]:
# Move the model to the appropriate device
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [8]:
file_path = '/content/dev.csv'
df_total = pd.read_csv(file_path)

In [9]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df_total, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size = 0.1111, random_state=42)

print(len(train), len(test), len(val))

3200 400 400


In [10]:
train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

In [11]:
train_dataset

Dataset({
    features: ['en_text', 'fa_text', '__index_level_0__'],
    num_rows: 3200
})

In [12]:
def preprocess_function(examples):
  inputs = examples["en_text"]
  targets = examples["fa_text"]

  model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")

  labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length', return_tensors="pt")

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [13]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

# Train Model

In [14]:
# Define Training Arguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart_results",
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    learning_rate=3e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='bleu'
)

In [15]:
bleu = evaluate.load("bleu")

In [16]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred

  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = [[(l if l!=-100 else tokenizer.pad_token_id) for l in label]for label in labels]

  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])

  return result

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [18]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length
500,0.319100,0.438741,0.064806,"[0.32374727668845316, 0.1054892601431981, 0.04069309530060383, 0.014795474325500435]",0.962382,0.963072,4590,4766
1000,0.225800,0.446908,0.065030,"[0.33456924754634676, 0.10967741935483871, 0.04180909808046279, 0.013650885855358699]",0.961292,0.962023,4585,4766


Trainer is attempting to log a value of "[0.32374727668845316, 0.1054892601431981, 0.04069309530060383, 0.014795474325500435]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Trainer is attempting to log a value of "[0.33456924754634676, 0.10967741935483871, 0.04180909808046279, 0.013650885855358699]" of type <class 'list'> for key "eval/precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Some non-default ge

TrainOutput(global_step=1200, training_loss=0.6298386875788371, metrics={'train_runtime': 1179.6984, 'train_samples_per_second': 8.138, 'train_steps_per_second': 1.017, 'total_flos': 2600555721523200.0, 'train_loss': 0.6298386875788371, 'epoch': 3.0})

In [19]:
def translate(sentence):


  encoded_input=tokenizer(sentence, return_tensors="pt", max_length=128, truncation=True).to(device)

  generated_tokens = model.generate(**encoded_input, max_length=128, num_beams=4, early_stopping=True, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])

  translated = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

  return translated

In [20]:
english_sentence= "How are you"
persian_translation = translate(english_sentence)

print("english sentence: ", english_sentence)
print("persian translation: ", persian_translation)

english sentence:  How are you
persian translation:  حالتون چطوره؟


In [21]:
trainer.save_model('./trained_mbart_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


In [22]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [23]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [28]:
sentence = "How are you"

In [29]:
# translate Arabic to English
tokenizer.src_lang = "en_XX"
encoded_ar = tokenizer(sentence, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["fa_IR"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


['چطوری ؟']